## Запрос во views для извлечения количества активных юхеров за каждый месяц.

In [2]:
import msgpack
import boto3
import awswrangler as wr
from io import BytesIO
import pickle as pkl
import numpy as np
import re
import datetime

import matplotlib.pyplot as plt
import json

np.random.seed(1)

# Connection

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
ACCESS_KEY='du3wAsX0uzEBBn4AdkFpCX4ZTXvSWFVG48GA4jbT'
SECRET_KEY='WA0NJOX0AFAY4K1J92XZ'

wr.config.s3_endpoint_url = 'https://obs.ru-moscow-1.hc.sbercloud.ru'

boto3.setup_default_session(
    region_name='ru-moscow',
    aws_access_key_id=SECRET_KEY,
    aws_secret_access_key=ACCESS_KEY,
)

def _get_default_boto3_client():
    return boto3.client('s3', endpoint_url=wr.config.s3_endpoint_url)

def get_object(s3_url):
    s3_client = _get_default_boto3_client()
    bucket, key = _split_3s_url_to_bucket_and_key(s3_url)
    return s3_client.get_object(Bucket=bucket, Key=key)

In [5]:
spark = SparkSession.builder.appName("Dummy").master('local').getOrCreate()
print("session = ", spark.getActiveSession())
config_spark_s3_adv = {'access_id': 'JLTHDJMKGTSENNZLD5YC',
                        'access_key': 'gtlSwf2xuppeomzYaAN3OFpcJiWtgAGUWNU7KIvJ',
                        'impl': 'org.apache.hadoop.fs.s3a.S3AFileSystem',
                        'endpoint': 'https://obs.ru-moscow-1.hc.sbercloud.ru'}

spark._jsc.hadoopConfiguration().set("fs.s3a.impl", config_spark_s3_adv["impl"])
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", config_spark_s3_adv["endpoint"])
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", config_spark_s3_adv["access_id"])
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", config_spark_s3_adv["access_key"])
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

22/10/24 11:45:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/24 11:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/24 11:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


session =  <pyspark.sql.session.SparkSession object at 0x7f8d9e30a980>


# Data

In [6]:
path_views = f's3://okko-prod/pipelines/etl-views_0.1/'

dirs = wr.s3.list_directories(path_views)

In [7]:
dirs

['s3://okko-prod/pipelines/etl-views_0.1/202206062359/',
 's3://okko-prod/pipelines/etl-views_0.1/202209072300/',
 's3://okko-prod/pipelines/etl-views_0.1/202209102146/',
 's3://okko-prod/pipelines/etl-views_0.1/202209110938/',
 's3://okko-prod/pipelines/etl-views_0.1/202209112219/',
 's3://okko-prod/pipelines/etl-views_0.1/202209121928/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130741/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130808/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130839/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130911/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130918/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130929/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130934/',
 's3://okko-prod/pipelines/etl-views_0.1/202209130937/',
 's3://okko-prod/pipelines/etl-views_0.1/202209131317/',
 's3://okko-prod/pipelines/etl-views_0.1/202209131421/',
 's3://okko-prod/pipelines/etl-views_0.1/202209131618/',
 's3://okko-prod/pipelines/etl-

In [8]:
path_views = 's3a://okko-prod/pipelines/etl-views_0.1/202209240504/outgoing/views.parquet/'
full_views = spark.read.format('parquet').load(path_views)

22/10/24 11:45:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [9]:
full_views.printSchema()

root
 |-- user_uid: string (nullable = true)
 |-- element_uid: string (nullable = true)
 |-- watched_time: long (nullable = true)
 |-- consumption_mode: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- price: float (nullable = true)



In [10]:
full_views.show()

+--------------------+--------------------+------------+----------------+-------------------+-----+
|            user_uid|         element_uid|watched_time|consumption_mode|                 ts|price|
+--------------------+--------------------+------------+----------------+-------------------+-----+
|00000000-0001-0eb...|a1f0d38e-6e75-452...|          11|               S|2022-07-24 18:40:51|  0.0|
|00000000-0001-0eb...|7b01f074-b318-46b...|         413|               S|2022-06-28 19:55:00|  0.0|
|00000000-0001-0eb...|736ffee7-f210-4ff...|        1164|               S|2022-07-14 20:12:49|  0.0|
|00000000-0001-0eb...|5b316b46-d764-4d0...|         967|               S|2022-01-15 16:27:13|  0.0|
|00000000-0001-0eb...|50adaca1-09c0-41e...|        1829|               S|2022-07-24 18:10:06|  0.0|
|00000000-0001-0eb...|6540228d-6f03-4e2...|         966|               S|2022-01-30 18:00:53|  0.0|
|00000000-0001-0eb...|ba0c76a9-371f-470...|         149|               S|2022-07-02 19:04:10|  0.0|


In [11]:
full_views.createOrReplaceTempView("views")

In [ ]:
query = f"""
            SELECT
                MIN(ts) AS min_ts,
                MAX(ts) AS max_ts
            FROM
                views
            """

dates = spark.sql(query)
dates.show()

In [30]:
query = f"""
            SELECT
                YEAR(ts) as year,
                MONTH(ts) AS month,
                COUNT(DISTINCT user_uid) AS user_count
            FROM
                views
            GROUP BY year, month
            """

dates = spark.sql(query)
dates.show()

+----+-----+----------+
|year|month|user_count|
+----+-----+----------+
|2021|    8|   1578018|
|2021|    6|   1170197|
|2021|    5|   1461383|
|2021|   10|   1827201|
|2020|   11|   1550071|
|2022|    2|   2304421|
|2022|    7|   1901659|
|2021|   11|   1852366|
|2021|    9|   1630587|
|2022|    3|   2433983|
|2020|   12|   1951784|
|2021|   12|   2128961|
|2021|    7|   1208495|
|2021|    3|   1505941|
|2021|    2|   1601893|
|2022|    1|   2739955|
|2022|    5|   2508235|
|2021|    1|   2154595|
|2020|   10|    372516|
|2022|    6|   1950860|
+----+-----+----------+
only showing top 20 rows



In [32]:
dates_pd = dates.toPandas()

In [33]:
dates_pd

,year,month,user_count
0,2021,8,1578018
1,2021,6,1170197
2,2021,5,1461383
3,2021,10,1827201
4,2020,11,1550071
5,2022,2,2304421
6,2022,7,1901659
7,2021,11,1852366
8,2021,9,1630587
9,2022,3,2433983


In [38]:
dates_pd.sort_values('user_count')

,year,month,user_count
18,2020,10,372516
1,2021,6,1170197
12,2021,7,1208495
2,2021,5,1461383
13,2021,3,1505941
22,2021,4,1510243
4,2020,11,1550071
0,2021,8,1578018
14,2021,2,1601893
8,2021,9,1630587


In [34]:
import pickle 

with open('active_users_months.pkl', 'wb') as f:
    pickle.dump(dates_pd, f)

In [13]:
query = f"""
            SELECT
                COUNT(DISTINCT element_uid)
            FROM
                views
            """

el_num = spark.sql(query)
el_num.show()

+---------------------------+
|count(DISTINCT element_uid)|
+---------------------------+
|                     168390|
+---------------------------+

